In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import regularizers
from keras.callbacks import CSVLogger
#from livelossplot import PlotLossesKeras
import os
import numpy as np
#from imgaug import augmenters as iaa
#import cv2
from keras.layers.normalization import BatchNormalization
import seaborn as sns
import pandas as pd 
from keras import initializers
from keras import optimizers
import keras.backend as K
import tensorflow as tf
from keras.models import load_model
from keras.models import Model
import cv2

from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

In [ ]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")
x_val = np.load("x_val.npy")
y_val = np.load("y_val.npy")
x_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")

In [ ]:
x_train = x_train / 255
x_val = x_val / 255
x_test = x_test/255

In [ ]:
IMAGE_SIZE = 200
IMAGE_WIDTH, IMAGE_HEIGHT = IMAGE_SIZE, IMAGE_SIZE
EPOCHS = 20
BATCH_SIZE = 64
TEST_SIZE = 30
input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(256))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2))
model.add(Activation('softmax'))



In [ ]:
from keras.optimizers import SGD
opt = SGD(lr=0.01)
model.compile(loss = "binary_crossentropy", optimizer = opt,  metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(x=x_train, y=y_train, validation_data = (x_val,y_val), batch_size = BATCH_SIZE, epochs=EPOCHS, verbose=1)

In [ ]:
model.evaluate(x=x_test, y=y_test, verbose=1)

In [ ]:
layer_name = 'dense_13'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)


In [ ]:
def tensor_3to4(img):
    '''
    Resizes and expands dimension
    
    Input: img: a 3-channel image as input
    
    Returns a rank-4 tensor, since the network accepts batches of images
    One image corresponds to batch size of 1
    '''
    img = cv2.resize(img, (200, 200))
    img_4d = np.expand_dims(img, axis=0)  # rank 4 tensor for prediction
    return img_4d

In [ ]:
x_features = []
for x in x_train:
    x = tensor_3to4(x)
    intermediate_output = intermediate_layer_model.predict(x)
    x_features.append(intermediate_output)

In [ ]:
len(x_features)

In [ ]:
x_features = np.array(x_features)
nsamples, nx, ny = x_features.shape
d2_train_dataset = x_features.reshape((nsamples,nx*ny))

In [ ]:
results = []

In [ ]:
classifier = SVC( probability=True, gamma=.1)
cv_score = cross_val_score(classifier, d2_train_dataset, y_train[:,1:2], cv=5, scoring='roc_auc')
results.append(cv_score)
classifier.fit(d2_train_dataset, y_train[:,1:2])

In [ ]:
cv_score

In [ ]:
test_features = []
for y in x_test:
    y = tensor_3to4(y)
    intermediate_output = intermediate_layer_model.predict(y)
    test_features.append(intermediate_output)

In [ ]:
test_features = np.array(test_features)
nsamples, nx, ny = test_features.shape
d2_test_dataset = test_features.reshape((nsamples,nx*ny))

In [ ]:
y_pred = classifier.predict(d2_test_dataset)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test[:,1:2], y_pred)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test[:,1:2], y_pred))

In [ ]:
from sklearn import metrics

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef

In [ ]:
matthews_corrcoef(y_test[:,1:2], y_pred)